In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('spam_ham_dataset.csv')

In [5]:
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [9]:
import string
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yashM\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [13]:
df['text'] =  df['text'].apply(lambda x: x.replace('\r\n',' '))

In [14]:
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291 thi...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001 ( see at...",0
2,3624,ham,"Subject: neon retreat ho ho ho , we ' re aroun...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs this deal is to b...,0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5171 non-null   int64 
 1   label       5171 non-null   object
 2   text        5171 non-null   object
 3   label_num   5171 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.7+ KB


In [16]:
stemmer = PorterStemmer()

In [17]:
corpus = []
stopword_set = set(stopwords.words('english'))

for i in range(len(df)):
    text = df['text'].iloc[i].lower()
    text = text.translate(str.maketrans('','',string.punctuation)).split()
    text = [stemmer.stem(word) for word in text if word not in stopword_set]
    text = ' '.join(text)
    corpus.append(text)

In [18]:
corpus[0]

'subject enron methanol meter 988291 follow note gave monday 4 3 00 preliminari flow data provid daren pleas overrid pop daili volum present zero reflect daili activ obtain ga control chang need asap econom purpos'

In [19]:
df.text.iloc[0]

"Subject: enron methanol ; meter # : 988291 this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary flow data provided by daren } . please override pop ' s daily volume { presently zero } to reflect daily activity you can obtain from gas control . this change is needed asap for economics purposes ."

In [20]:
vectorizer = CountVectorizer()

In [21]:
X = vectorizer.fit_transform(corpus).toarray()
y = df.label_num

In [22]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

##  random forest

In [23]:
clf = RandomForestClassifier(n_jobs=-1)

clf.fit(X_train,y_train)

RandomForestClassifier(n_jobs=-1)

In [24]:
clf.score(X_test,y_test)

0.978743961352657

### Testing 

In [25]:
email_to_clasify = df.text.iloc[3]

In [52]:
def text_vector(text_to_convert):
    email_text = text_to_convert.lower()
    email_text = email_text.translate(str.maketrans('','',string.punctuation)).split()
    email_text = [stemmer.stem(word) for word in email_text if word not in stopword_set]
    email_text = ' '.join(email_text)
    # print(email_text)
    return vectorizer.transform([email_text])

In [49]:
clf.predict(text_vector(email_to_clasify))

subject photoshop window offic cheap main trend abas darer prudent fortuit undergon lightheart charm orinoco taster railroad affluent pornograph cuvier irvin parkhous blameworthi chlorophyl robe diagrammat fogarti clear bayda inconvenienc manag repres smart hashish academi sharehold unload bad danielson pure caffein spaniard chargeabl levin


array([1], dtype=int64)

In [50]:
clf.predict(text_vector(df.text.iloc[0]))[0]

subject enron methanol meter 988291 follow note gave monday 4 3 00 preliminari flow data provid daren pleas overrid pop daili volum present zero reflect daili activ obtain ga control chang need asap econom purpos


0

In [61]:
e = 0
n = len(df)
for i in range(n):
    e += 1 if clf.predict(text_vector(df.text.iloc[i]))[0] != df.label_num.iloc[i] else 0
    # print(i)
print(e/n)
    

0.0042544962289692515


In [51]:
text_vector(df.text.iloc[1])

subject hpl nom januari 9 2001 see attach file hplnol 09 xl hplnol 09 xl


<1x42637 sparse matrix of type '<class 'numpy.int64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [44]:
df.text.iloc[1]

'Subject: hpl nom for january 9 , 2001 ( see attached file : hplnol 09 . xls ) - hplnol 09 . xls'

22.0

## Save model

In [57]:
import joblib

In [60]:
joblib.dump(vectorizer,'vectorizer_model.joblib' )

['vectorizer_model.joblib']

In [62]:
joblib.dump(clf,'model.joblib')

['model.joblib']